In [2]:
import sys
sys.path.append('/Users/shayecarver/residPCA/src/')
from residPCA import *

Morab_RNA = residPCA(count_matrix_path="/Users/shayecarver/condPCA/Morabito/RNA/data/Morabito_RNA_QC_RAW_COUNTS.h5ad", 
               vars_to_regress=['Batch','celltype','total_counts', 'pct_counts_mt','Age','Sex'],
               object_columns=['celltype','Sex','Batch'], 
               save_image_outputs=False, 
               BIC=True, 
               vargenes_Stand_resid=3000,
               n_PCs = 150,
               random_seed=7, 
               vargenes_IterPCA=3000, 
               #path_to_directory = "./", 
               #basename=f'Morab_RNA_3kvargenes_seurat',
               variable_genes_flavor="seurat",  
               global_ct_cutoff=0.2)  
Morab_RNA.Normalize() 
Morab_RNA.Standardize()
Morab_RNA.StandardPCA_fit()
Morab_RNA.residPCA_fit()
Morab_RNA.Iter_PCA_fit()
# identify global versus cell type specific states
Morab_RNA.ID_Global_CellType_States()


Finding most variable genes using 'seurat' flavor.
Using "seurat" flavor to compute variable genes in Iterative PCA on ASC.
Using "seurat" flavor to compute variable genes in Iterative PCA on EX.
Using "seurat" flavor to compute variable genes in Iterative PCA on INH.
Using "seurat" flavor to compute variable genes in Iterative PCA on MG.
Using "seurat" flavor to compute variable genes in Iterative PCA on ODC.
Using "seurat" flavor to compute variable genes in Iterative PCA on OPC.
Using "seurat" flavor to compute variable genes in Iterative PCA on PER.END.
